In [11]:
import pandas as pd
import ast
import numpy as np

In [41]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)


In [12]:
post_with_topic = pd.read_csv("../topic-modeling/data/positive_topic_dist.csv")
print(post_with_topic.columns)
post_with_topic["Topic Distribution"] = post_with_topic["Topic Distribution"].apply(lambda x: ast.literal_eval(x))
post_with_topic.head()

Index(['Unnamed: 0', 'subreddit_id', 'author', 'selftext', 'title', 'id',
       'created_utc', 'Topic Distribution'],
      dtype='object')


,Unnamed: 0,subreddit_id,author,selftext,title,id,created_utc,Topic Distribution
0,1,t5_o3plh,-CreamyPie-,Hello!\n\nIm 11 and Im a boy self recovering f...,Fear Food Friday!! So far its successful,gbj0rh,1588343070,"[0.11385973850293842, 0.3352082928986615, 0.20..."
1,1,t5_o3plh,-CreamyPie-,Alright so Its wednesday the day im typing thi...,Fear Food Friday is almost here.,gagddu,1588190127,"[0.11385973850293842, 0.47646002034584006, 0.1..."
2,1,t5_o3plh,-CreamyPie-,"Im 11, Im a boy, you've probably seen my other...",Smoothie!!,g7xk05,1587836033,"[0.020330066198061984, 0.5405101500269072, 0.2..."
3,22,t5_s5o7i,194569324,Just want to get this off my chest because I d...,"Hardcore restricting, close to being discharge...",g3255g,1587130632,"[0.08260721895128373, 0.00473008524034919, 0.4..."
4,22,t5_s5o7i,194569324,"I feel horrible because I love him so much, bu...",My boyfriend triggers my eating disorder.,fvycov,1586178189,"[0.01233986429530161, 0.4562169014396947, 0.32..."


In [13]:
topic_list = [
    'NUM treatment get work need year time therapist last back', #0
 'eat food eating day meal hungry like meals calories binge',    #1
 'life recovery people body self nNUM things would every love',  #2 
 'look body like weight fat hate see skinny people always',
 'NUM eat ate chocolate foods sugar cream food cake one',
 'NUM weight gain back eating months lose lost still body',
 'like know feel want really even eating get think never',
 'said told didn mom family made got would one went',
 'doctor blood take heart symptoms hospital taking doctors medication pain',
 'like NUM feel day want get today time days eat']

In [14]:
post_with_topic["most_probable_topic"] = post_with_topic["Topic Distribution"].apply(lambda x: np.array(x).argmax())
post_with_topic["most_probable_topic"].value_counts()

2    1876
9     253
1     177
4     141
6      98
7      79
8      64
5      60
3      53
0      39
Name: most_probable_topic, dtype: int64

In [28]:
trigger_extraction_result = pd.read_csv("./results/trigger_extraction_result_1682559339.705567.csv", keep_default_na = False)
for i in range(3):
    trigger_extraction_result[f"trigger_{i}"] = trigger_extraction_result[f"trigger_{i}"].apply(lambda x: ast.literal_eval(x) if x.startswith("[") else (None if x == "" else [x]))
    print(trigger_extraction_result[f"trigger_{i}"].explode().value_counts(dropna=False))
    

None                                                         137
stress                                                         3
gaining weight                                                 2
losing weight                                                  2
relapse                                                        2
                                                            ... 
physical affection from friend                                 1
birthday                                                       1
mom ignoring signs                                             1
seeing sibling lose weight                                     1
recent weight gain after pregnancy and fight with partner      1
Name: trigger_0, Length: 307, dtype: int64
None                                       141
depression                                   3
gaining weight                               2
stress                                       2
anxiety                                      2
       

In [29]:
trigger_extraction_result = trigger_extraction_result.merge(post_with_topic[["id","most_probable_topic"]], on="id", how="left")

In [43]:
trigger_extraction_result["most_probable_topic"].value_counts(dropna=False)

NaN    294
2.0      4
3.0      1
4.0      1
Name: most_probable_topic, dtype: int64

In [42]:
def most_common(x):
    top_5 = x.explode().value_counts(dropna=False)[:5]
    return [(x, c) for x, c in zip(top_5.index, top_5)]
trigger_extraction_result.groupby(["most_probable_topic"]).agg({
    "trigger_0": [most_common],
    "trigger_1": [most_common],
    "trigger_2": [most_common]
})

,trigger_0,trigger_1,trigger_2
,most_common,most_common,most_common
most_probable_topic,,,
2.0,"[(None, 2), (eating cake at birthday parties, 1), (stomach bloating after eating, 1), (winter, 1)]","[(None, 3), (stomach bloating after eating, 1)]","[(None, 3), (stomach bloating after eating, 1)]"
3.0,"[(stopped taking laxatives, 1), (suffering with backed up bloat and discomfort, 1)]","[(stopped taking laxatives, 1), (suffering with the backed up bloat and discomfort, 1)]","[(stopped taking laxatives, 1), (suffering with backed up bloat and discomfort, 1)]"
4.0,"[(she is meeting with the girl she changed me with next Wednesday and I can't shake the thought that she preferred her over me., 1)]","[(finding out that girlfriend made out with someone else, 1), (girlfriend meeting with the person she cheated on the writer with, 1)]","[(she is meeting with the girl she cheated on me with next Wednesday, 1)]"


In [9]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

## Create input file based on topic modeling

In [45]:
full_narrative_df = pd.read_csv("../narrative_detection/annotated_narrative_posts_by_trained_classification.csv")
narrative_with_topic_model = full_narrative_df.merge(post_with_topic[["id","most_probable_topic"]], on="id", how="inner")
print(len(narrative_with_topic_model))
print(narrative_with_topic_model.columns)


2839
Index(['Unnamed: 0', 'selected_for_experiment', 'subreddit', 'selftext', 'author_fullname', 'title', 'score', 'link_flair_css_class', 'link_flair_text', 'author_flair_type', 'over_18', 'author_flair_text', 'subreddit_id', 'id', 'is_self', 'author', 'author_flair_css_class', 'permalink', 'url', 'created_utc', 'num_words', 'narrative_positive', 'narrative_negative', 'narrative_uncertain', 'narrative_label', 'predicted_narrative_label', 'most_probable_topic'], dtype='object')


In [51]:
np.random.seed(20)
experiment_index = []
for topic_i in range(len(topic_list)):
    print(f"Topic {topic_i}")
    topic_all_indexes = narrative_with_topic_model[narrative_with_topic_model["most_probable_topic"] == topic_i].index
    sample_size = min(50, len(topic_all_indexes))
    selected_indexes = np.random.choice(topic_all_indexes, sample_size, replace = False)
    print(selected_indexes)
    experiment_index.extend(selected_indexes)

Topic 0
[ 387 1984 1125  141 2215  923 1763  770  384 1360  657 1491 2500 2794
 2111  462 2716 2281 1777  493 1313 1569 1622  567 2699  103  383  559
 1714  690 1609  637 2793 2117 2409 1247 2056 2792]
Topic 1
[  83  765 2670 2199 2277  351  838 2465 1756 2514  445 1950  286 1402
 2377 1998    9 1420  893 2471  869 2470  950 1793 2383  777 2003 1590
 1285  643  592 2467 2163 2784 2633 1537 1378 1623 1991 1369   90 1414
 2469 2776 1169 1761 1001  908  891 2472]
Topic 2
[ 713 2817  362 1044 1915  924  210 2542 1928   96  813 1811 1223 1721
  725  192  407 2755 2011 1118  378 1349  481 2493 2779  323 2440  465
  989 1858 1805 1792  564 1518 2620 1575 2799 2347 1437 1637 2606   84
 2544  361 2115 1635 2617 2454  317 1465]
Topic 3
[1029 1272 1280 1356 2585  927 2816 1691 1151 1060 2520 1250 2575 1238
 2507 2532 1576  918  207  683 2636 2570 2602 1072 1323 1083 2627 1264
  437 2642 2790 2654 1904 2557 2697 1115  985 1965  454 1548 2517 1686
    3 2526 2531  159 1901 2768 2624 1897]
Topic 4
[

In [73]:
narrative_with_topic_model.loc[experiment_index,"selected_for_experiment"] = 1

In [78]:
narrative_with_topic_model["selected_for_experiment"].value_counts(dropna=False)

NaN    2338
1.0     501
Name: selected_for_experiment, dtype: int64

In [79]:
narrative_with_topic_model.to_csv("../narrative_detection/narrative_post_with_topic_model.csv", index=False)